In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")

OpenAI Class: This class is typically used for the older models (like GPT-3 and earlier versions) that interact through the /v1/completions endpoint. These models process text input as a single prompt and return a text-based completion.

ChatOpenAI Class: This class is specifically designed for chat-based models like GPT-4, which interact through the /v1/chat/completions endpoint.These models are structured to handle conversations, where input is provided as a series of messages (as a conversation) rather than a single prompt.

I used ChatopenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo")

REACT_DOCSTORE is designed to work with exactly two tools: a search tool and a lookup tool. 

Because of this restriction I used more flexible tool like: ZERO_SHOT_REACT_DESCRIPTION

In [ ]:
docstore = DocstoreExplorer(Wikipedia())

def simple_math(question: str) -> str:
    try:
        answer = eval(question)
        return str(answer)
    except Exception as e:
        return str(e)

math_tool = Tool(
        name = "MathTool",
        func = simple_math,
        description= "useful for performing basic arithmetic calculations"
    )

tools =  [
    Tool(
        name = "Search",
        func = docstore.search,
        description="useful for when you need to ask with search"        
    ),
    Tool(
        name = "Lookup",
        func = docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

tools.append(math_tool)

react = initialize_agent(tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [ ]:
question = "What is 100/25"
react.run(question)